In [1]:
!pip install syft


In [2]:
import syft as sy
import torch as th

W0718 12:17:14.425172 140451384371072 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0718 12:17:14.443325 140451384371072 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [3]:
x = th.tensor([1,2,3,4])
x

tensor([1, 2, 3, 4])

In [0]:
hook = sy.TorchHook(th)

In [0]:
bob = sy.VirtualWorker(hook, id="bob")

In [6]:
bob._objects # simple tensors

{}

In [0]:
x= th.tensor([1,2,3,54])

In [0]:
x = x.send(bob)

In [9]:
bob._objects

{17024707784: tensor([ 1,  2,  3, 54])}

In [10]:
x.location #show the virtual location

<VirtualWorker id:bob #objects:1>

In [11]:
x.id

62131411232

In [12]:
x.owner

<VirtualWorker id:me #objects:0>

In [13]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [0]:
x=x.get()#get the tensor from bob

In [15]:
x

tensor([ 1,  2,  3, 54])

In [0]:
alice = sy.VirtualWorker(hook, id='alice')


In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x_ptr = x.send(bob, alice)

In [19]:
x_ptr.get()

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [0]:
x= th.tensor([1,2,3,4,5]).send(bob, alice)

In [21]:
x.get(sum_results=True)

tensor([ 2,  4,  6,  8, 10])

In [0]:
###

Introduction to Remote Arithmetic

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [24]:
x

(Wrapper)>[PointerTensor | me:82327760966 -> bob:91931435705]

In [0]:
z = x+y

In [26]:
z

(Wrapper)>[PointerTensor | me:11860664024 -> bob:75604122287]

In [27]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:22614772187 -> bob:50969232508]

In [0]:
z = z.get()


In [29]:
z

tensor([2, 3, 4, 5, 6])

In [0]:
x = th.tensor([1.,2,3,4,5],requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1],requires_grad=True).send(bob)

In [0]:
z = (x + y).sum()

In [32]:
z.backward()

(Wrapper)>[PointerTensor | me:89603693081 -> bob:50598547729]

In [0]:
x = x.get()

In [34]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [35]:
x.grad

tensor([1., 1., 1., 1., 1.])

Simple Linear Model

In [0]:
input = th.tensor([[1.,1], [0,1], [1,0], [0,0]], requires_grad=True).send(bob)
target = th.tensor([[1.], [1],[0], [0]], requires_grad=True).send(bob)

In [0]:
weights = th.tensor([[0.], [0.]], requires_grad=True).send(bob)

In [38]:
for i in range(10):
  pred = input.mm(weights)

  loss = ((pred - target)**2).sum()

  loss.backward()

  weights.data.sub_(weights.grad * 0.1)
  weights.grad *= 0
  print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


Garbage Collections and Common Error [Practice] 

In [0]:
bob = bob.clear_objects()

In [40]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [42]:
bob._objects

{57818546686: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [44]:
bob._objects

{}

In [0]:
x = "asksd"

In [46]:
bob._objects

{}

In [0]:
del x

In [48]:
bob._objects

{}

In [49]:
bob = bob.clear_objects()
bob._objects

{}

In [0]:
for i in range(1000):
  x = th.tensor([1,3,3,3,]).send(bob)
  

In [51]:
bob._objects

{3525517415: tensor([1, 3, 3, 3])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob) 
y = th.tensor([1,2,3,4,5]).send(alice)

In [53]:
The above two pointers x & y are errors as you can't locate two different tensors on same machine
z = x + y

SyntaxError: ignored

Toy Federated Learnings

In [0]:
from torch import nn, optim

In [0]:
#toy dataset
data = th.tensor([[1.,1], [0,1], [1,0], [0,0]], requires_grad=True)
target = th.tensor([[1.], [1], [0], [0]], requires_grad=True)

In [0]:
# A toy model
model = nn.Linear(2,1)

In [0]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [58]:
def train(iteration=20):
  for iter in range(iteration): 
    opt.zero_grad()
    pred = model(data)
    loss = ((pred-target)**2).sum()
    loss.backward()
    opt.step()
    print(loss.data)
train()

tensor(1.7808)
tensor(1.1514)
tensor(0.7822)
tensor(0.5380)
tensor(0.3731)
tensor(0.2608)
tensor(0.1838)
tensor(0.1305)
tensor(0.0934)
tensor(0.0673)
tensor(0.0488)
tensor(0.0356)
tensor(0.0262)
tensor(0.0193)
tensor(0.0143)
tensor(0.0107)
tensor(0.0080)
tensor(0.0060)
tensor(0.0045)
tensor(0.0034)


In [0]:
data_bob = data[0:2].send(bob)
target_bob= target[0:2].send(bob)

In [0]:
data_alice = data[0:2].send(alice)
target_alice = target[0:2].send(alice)

In [0]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [0]:
def train(iterate=20):
  model = nn.Linear(2,1)
  opt = optim.SGD(params=model.parameters(), lr=0.1)

  for iter in range(iterate):
    _data, _target = datasets[0]
    #send the model to data
    model = model.send(_data.location)

    #do normal training
    opt.zero_grad()
    pred = model(_data)
    loss = ((pred - _target)**2).sum()
    loss.backward()
    opt.step()

      #gets smarter model
    model = model.get()
    print(loss.get())


In [89]:
train()

tensor(3.5936, requires_grad=True)
tensor(0.0677, requires_grad=True)
tensor(0.0339, requires_grad=True)
tensor(0.0280, requires_grad=True)
tensor(0.0233, requires_grad=True)
tensor(0.0194, requires_grad=True)
tensor(0.0162, requires_grad=True)
tensor(0.0135, requires_grad=True)
tensor(0.0112, requires_grad=True)
tensor(0.0093, requires_grad=True)
tensor(0.0078, requires_grad=True)
tensor(0.0065, requires_grad=True)
tensor(0.0054, requires_grad=True)
tensor(0.0045, requires_grad=True)
tensor(0.0037, requires_grad=True)
tensor(0.0031, requires_grad=True)
tensor(0.0026, requires_grad=True)
tensor(0.0021, requires_grad=True)
tensor(0.0018, requires_grad=True)
tensor(0.0015, requires_grad=True)


In [64]:
_data.location # client

<VirtualWorker id:bob #objects:6>

Advanced Remote Execution

In [90]:
bob.clear_objects()

<VirtualWorker id:bob #objects:0>

In [91]:
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [96]:
x.send(alice) # pointer replaced instead of bob

(Wrapper)>[PointerTensor | me:2354920631 -> alice:77102619543]

In [95]:
bob._objects

{59763512361: tensor([1, 2, 3, 4, 5])}

In [97]:
alice._objects

{77102619543: (Wrapper)>[PointerTensor | alice:77102619543 -> bob:59763512361]}

In [0]:
y = x + x

In [99]:
y

(Wrapper)>[PointerTensor | me:13285567616 -> bob:5367576295]

In [100]:
bob._objects

{5367576295: tensor([ 2,  4,  6,  8, 10]),
 59763512361: tensor([1, 2, 3, 4, 5])}

In [101]:
alice._objects

{77102619543: (Wrapper)>[PointerTensor | alice:77102619543 -> bob:59763512361]}

In [0]:
# create a syft virtualWorker
jon = sy.VirtualWorker(hook, id="jon")

In [0]:
bob.clear_objects()
#alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)
y = th.tensor([1,2,3,4,5]).send(bob).send(jon)

In [118]:
x = x.get() #bob has the actual data 
x # pointer from bob to alice
# Now Alice has a tensor of bob

(Wrapper)>[PointerTensor | me:54607790466 -> bob:39542283832]

In [119]:
bob._objects

{39542283832: tensor([1, 2, 3, 4, 5]), 62383246799: tensor([1, 2, 3, 4, 5])}

In [120]:
alice._objects # Federated Learning Prevents Memory Leak

{}

**Pointer Chain Operation**

In [121]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [123]:
bob._objects

{4287163235: tensor([1, 2, 3, 4, 5])}

In [124]:
alice._objects

{93653932467: (Wrapper)>[PointerTensor | alice:93653932467 -> bob:4287163235]}

In [126]:
#inline command
x.remote_get()

(Wrapper)>[PointerTensor | me:74616317773 -> alice:93653932467]

In [127]:
bob._objects # bob has nothing to its machine

{}

In [128]:
alice._objects #alice recievs data from bob machine

{93653932467: tensor([1, 2, 3, 4, 5])}

In [129]:
x.move(bob)

(Wrapper)>[PointerTensor | me:74616317773 -> bob:74616317773]

In [130]:
x

(Wrapper)>[PointerTensor | me:74616317773 -> bob:74616317773]

In [131]:
bob._objects # data moved to bob's machine by remote_get

{74616317773: tensor([1, 2, 3, 4, 5])}

In [132]:
alice._objects #Alice has nothing to its machine


{}